In [ ]:
import numpy as np
import multiprocessing as mp
import time

V_MIN =  10.0
V_MAX =  42.0
A_MAX = 100.0
RADIUS = 1.0
a0 = 15 #Für die Geschwindigkeitsregulation


class Box:

    def __str__(self):
        return f"Box(xmin={self.xmin}, xmax={self.xmax}, "\
                   f"ymin={self.ymin}, ymax={self.ymax})"

    def get_x_limits(self):
        return (self.xmin, self.xmax)

    def get_y_limits(self):
        return (self.ymin, self.ymax)


class Puck:

    RADIUS = 1.0

    def __str__(self):
        return f"Puck(Id={self.id}, Alive={self.alive}, "\
               f"s={self.s}, v={self.v}, a={self.a}"

    def is_alive(self):
        return self.alive

    def get_id(self):
        return self.id

    def get_time(self):
        return self.t

    def get_position(self):
        return self.s

    def get_velocity(self):
        return self.v

    def get_acceleration(self):
        return self.a

    def get_name(self):
        return self.name

    def get_fuel(self):
        return self.fuel

    def get_points(self):
        return self.points


#Bubble_Sort sortiert die Liste der Beschleunigungswerte. Der höchste Wert ist an erster Stelle der Liste
#Das muss vor dem Abgleich mit der Maximalbeschleunigung passieren
#Es soll immer die Norm der Beschleunigungsvektoren verglichen werden.
def bubble_sort(liste):
    """Eine Liste an Zahlen wird nach absteigendem Wert sortiert"""
    for i in range(len(liste)):
        for j in range(0,len(liste)- i - 1):
            if np.linalg.norm(liste[j]) < np.linalg.norm(liste[j+1]):
                l1 = liste[j]
                liste[j] = liste[j+1]
                liste[j+1] = l1
    return liste





def worker_brinkmann(id, secret, q_request, q_reply):
    #Vorbereitung: Set_Name und Identifikation der Nummer
    q_request.put(('SET_NAME', "Philipp", secret, id))
    
    reply = q_reply.get()
    q_request.put(('GET_SIZE', id))
    reply1 = q_reply.get()
    N = reply1[1] #Spieleranzahl
    for i in range(0,N): #Ermittlung der eigenen Nummer
        q_request.put(('GET_PUCK', i, id))
        reply2 = q_reply.get()
        puck = reply2[1]
        if puck.get_id() == id:
            n = i #Meine eigene Nummer
            break
#    print(n)

    
        #Start der Ausweichprozesse
    while True:
        liste_der_ausgeschiedenen_mitspieler = []
        liste_der_beschleunigungswerte = []
        liste_der_beschleunigungswerte.clear()
        q_request.put(('GET_PUCK', n, id))
        reply3 = q_reply.get()
        puck = reply3[1]
#        print(f"{puck.get_fuel()}, Puck{n}")

        
        #Vorbereitung Beschleunigungsberechnung
        r = puck.get_position()
        v = puck.get_velocity()
        if puck.is_alive():
            for j in range(0, N):
                if j == n:
                    j += 1
                elif j in liste_der_ausgeschiedenen_mitspieler:
                    j += 1
                else:
                    q_request.put(('GET_PUCK', j, id))
                    reply4 = q_reply.get()
                    puck2 = reply4[1]
                    if puck2.is_alive() == True:
                        r1 = puck2.get_position()
                        v1 = puck2.get_velocity()
    
                        delr = r1 - r
                        delv = v1 - v
                        tca = -((np.dot(delr,delv))/np.dot(delv,delv))
                        rca = delr + tca*delv
                        rca_länge = np.linalg.norm(rca)
                        if tca < 0 or rca_länge >= 5*RADIUS: #2*R ist der Grenzfall: 5*R verschafft Toleranz
                            res_a = np.array([0, 0]) #Negative Zeiten sollen vernachlässigt werden, da die Pucks anscheinend schon aneinander vorbeigeflogen sind.
#                            print(f"Kein Ausweichmanöver, Puck{n}")
                        else:
                            a = np.linalg.norm((2*(2*(RADIUS + 1.0) -rca))/tca**2) #+1 Für Toleranz
                            #print(f"a = {a}") = Beschleunigung, die zum Ausweichen benötigt wird
                            #Die benötigte Beschleunigung muss überprüft werden
                            if a > A_MAX:
                                a = A_MAX - 1.0
                            res_a = (a/np.linalg.norm(rca))*(-rca)
                            liste_der_beschleunigungswerte.append(res_a)
                            j += 1
                    else:
                        liste_der_ausgeschiedenen_mitspieler.append(j)
                        j += 1
    
                
            #Errechnete Beschleunigungen priorisieren durch Bubble_Sort  
            list_sorted = bubble_sort(liste_der_beschleunigungswerte)
#                print(f"{list_sorted}, Puck{n}")
            if len(list_sorted) >= 1:
                q_request.put(('SET_ACCELERATION', list_sorted[0], secret, id))
                reply5 = q_reply.get()
#                    print(f"Beschleunigung zum Ausweichen: {reply5[1]}, Puck{n}")

            
            #Geschwindigkeitskontrolle
            q_request.put(('GET_PUCK', n, id))
            reply6 = q_reply.get()
            own_puck = reply6[1]
            velocity = own_puck.get_velocity()
            while np.linalg.norm(velocity) < 20.0 or np.linalg.norm(velocity) > 30.0:
                v0 = own_puck.get_velocity()
                if own_puck.is_alive() == False:
                    break
                elif np.linalg.norm(v0) < 20.0:
                    q_request.put(('SET_ACCELERATION', a0*v0/np.linalg.norm(v0), secret, id))
                    reply7 = q_reply.get()
#                        print(f"Anpassung Gas, Puck{n}")
                elif np.linalg.norm(v0) > 30.0:
                    q_request.put(('SET_ACCELERATION', a0*(-v0)/np.linalg.norm(v0), secret, id))
                    reply8 = q_reply.get()
#                        print(f"Anpassung Bremsen, Puck{n}")
                
            q_request.put(('SET_ACCELERATION', np.array([0,0]), secret, id))
            reply9 = q_reply.get()
#                print(f"Beschleunigungsreset, Puck{n}")


        
        else: #Wenn der Puck ausgeschieden ist, soll das Programm abbrechen
            break

